# # <div style="text-align:center"> Τεχνικές Εξόρυξης Δεδομένων: 2η Άσκηση  </div>

### <div style="text-align:center">  Παναγιώτοπουλος Γεώργιος **1115201700113** </div>

Αρχικά θα εισάγετε όλο το αρχείο σε ένα dataframe και θα μελετήσετε αν υπάρχουν τιμές Nan
στις στήλες ώστε να αφαιρέσετε αυτές τις γραμμες από το dataframe. Στην συνέχεια θα γράψετε
την κατάλληλες εντολές σε python για να απαντήσετε στα παρακάτω ζητούμενα. Τι περισσότερες
φορές οι απαντήσει θα δίνονται με ένα γράφημα και μπορείτε να χρησιμοποιήσετε όποια python
βιβλιοθήκη θέλετε για το σκοπό αυτό.

In [1]:
import os 
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

#df = pd.read_csv('./books_1.Best_Books_Ever.csv', nrows=50)
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/books_1.Best_Books_Ever.csv')
df.info(verbose=True)
print("Number of null values in 'title' column before cleaning:", df['series'].isnull().sum())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52478 entries, 0 to 52477
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            52478 non-null  object 
 1   title             52478 non-null  object 
 2   series            23470 non-null  object 
 3   author            52478 non-null  object 
 4   rating            52478 non-null  float64
 5   description       51140 non-null  object 
 6   language          48672 non-null  object 
 7   isbn              52478 non-null  object 
 8   genres            52478 non-null  object 
 9   characters        52478 non-null  object 
 10  bookFormat        51005 non-null  object 
 11  edition           4955 non-null   object 
 12  pages             50131 non-null  object 
 13  publisher         48782 non-null  object 
 14  publi

In [2]:
# Μπορούμε να βάλουμε τo median price σε όσα βιβλία δεν εχουν κάποια τίμη
import re
def fix_double_dots(string): #Some of the prices have three `.` for some reason
    pattern = r'^(\w+)\.(\w+)\.(\w+)$'
    match = re.match(pattern, string)
    if match:
        return match.group(1) + match.group(2) + '.' + match.group(3)
    else:
        return string
    
prices = []
for i in range(len(df.index)):
    if isinstance(df['price'][i],str):
        price = fix_double_dots(df['price'][i])
    else:
        price = df['price'][i]
    prices.append(float(price))      
df['price'] = prices
price_median = df['price'].median()
df['price'].fillna(price_median, inplace=True)

# Επίσης μπορόυμε να υποθέσουμε ότι για τα βιβλία που δεν αναγράφεται edition,ισχύει το Standard edition
df['edition'].fillna('Standard Edition',inplace=True)

# Και ότι για όσα δεν αναγράφεται Series, ΄ότι είναι Standalone
df['series'].fillna('Standalone',inplace=True)

df_clean = df.dropna().reset_index(drop=True)
df_clean.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27121 entries, 0 to 27120
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            27121 non-null  object 
 1   title             27121 non-null  object 
 2   series            27121 non-null  object 
 3   author            27121 non-null  object 
 4   rating            27121 non-null  float64
 5   description       27121 non-null  object 
 6   language          27121 non-null  object 
 7   isbn              27121 non-null  object 
 8   genres            27121 non-null  object 
 9   characters        27121 non-null  object 
 10  bookFormat        27121 non-null  object 
 11  edition           27121 non-null  object 
 12  pages             27121 non-null  object 
 13  publisher         27121 non-null  object 
 14  publishDate       27121 non-null  object 
 15  firstPublishDate  27121 non-null  object 
 16  awards            27121 non-null  object

## 1) Προεπεξεργασία (10%) 
Παρατηρήστε την στήλη ratingsByStars, περιέχει 5 τιμές χωρισμένες με κόμματα , χωρίστε τις
τιμές αυτές και προσθέστε στο dataframe ξεχωριστά τα ratings, δηλαδή ratingStar5, ratingStar4,
ratingStar3 κτλ.
Επίσης η στήλη genres περιέχει για κάθε βιβλίο περισσότερα από ένα genre (είδος).
Δημιουργήστε μία νέα στήλη (ονομάστε την genreSingle) και βάλτε μόνο το πρώτο genre από
όλα τα genres που συναντάμε σε κάθε γραμμή. (πχ ['Fantasy', 'Young Adult', 'Fiction', 'Magic',
'Childrens', 'Adventure', 'Audiobook', 'Middle Grade', 'Classics', 'Science Fiction Fantasy'] -> η
νέα στήλη θα έχει το 'Fantasy' ) deleting books without any genre information.
Χρησιμοποιήστε την στήλη publishDate και δημιουργήστε μία νέα στήλη με το έτος έκδοσης κάθε
βιβλίου (μπορείτε να χρησιμοποιήστε την μέθοδο to_datetime() που παρέχει το pandas ή ότι
άλλο θέλετε) .

In [3]:
import numpy as np
!pip install dateparser
import dateparser

#rows = len(df_clean.index)

# Useful function for getting rid of trash symbols
def remove_trash_symbols(string,symbols):
    for symbol in symbols:
            string = string.replace(symbol, "")
    return string

# Seperating rating entries
symbols_to_remove = "[]''"
clean = []
reformatted = []
for i in range(len(df_clean.index)): # For the ratings of every book
    entry = df_clean['ratingsByStars'][i]
    if(entry == "[]"): # some completely empty entries for some reason exist
        entry = "[0,0,0,0,0]"
    formatted = list(entry.split(",")) # split the string in the comas
    for rating in formatted: # and for every element of the broken form ['1593642 left
        string_left = remove_trash_symbols(rating,symbols_to_remove)
        clean.append(string_left)

reformatted = np.array(clean).reshape(len(df_clean.index),5)

# Adding the rows of the reformatted array into the dataframe
df_clean['ratingStar5'] = reformatted[:,0]
df_clean['ratingStar4'] = reformatted[:,1]
df_clean['ratingStar3'] = reformatted[:,2]
df_clean['ratingStar2'] = reformatted[:,3]
df_clean['ratingStar1'] = reformatted[:,4]

# Seperating genre
first_genre = []
for i in range(len(df_clean.index)): # For the ratings of every book
    entry = df_clean['genres'][i]
    first_genre.append(remove_trash_symbols(entry.split(',')[0],symbols_to_remove))
# Adding the new main genre column
df_clean['genreSingle'] = first_genre

years = []
for date in df_clean['publishDate']:
    parsed_date = dateparser.parse(date)
    if parsed_date is not None:
        years.append(parsed_date.year)
    else:
        years.append(None) 
df_clean['publishYear'] = years

df_clean = df_clean.dropna().reset_index(drop=True)
df_clean.info(verbose=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26889 entries, 0 to 26888
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            26889 non-null  object 
 1   title             26889 non-null  object 
 2   series            26889 non-null  object 
 3   author            26889 non-null  object 
 4   rating            26889 non-null  float64
 5   description       26889 non-null  object 
 6   language          26889 non-null  object 
 7   isbn              26889 non-null  object 
 8   genres            26889 non-null  object 
 9   characters        26889 non-null  object 
 10  bookFormat        26889 non-null  object 
 11  edition           26889 non-null  object 
 12  pages             26889 non-null  object 
 13  publisher         26889 non-null  object 
 14  publishDate       26889 non-null  o

## 2) Ερωτήματα για μελέτη των δεδομένων - απαντήστε σε 5 από τα παρακάτω (20%)
1. Κατασκευάστε το ιστόγραμμα των ratings στο σύνολο δεδομένων (χρησιμοποιήστε την
στήλη rating)
2. Ποιά είναι τα 10 βιβλία με τις περισσότερες σελίδες.
3. Ποιά είναι τα 10 βιβλία με τα περισσότερα 5-αστέρια (χρησιμοποιήστε μόνο τα βιβλία που
έχουν λάβει πάνω από 10.000 5-star ratings από τη στήλη ratingStar5) .
4. Ποιές είναι οι πιο συχνές λέξεις στους τίτλους των βιβλίων (αφού αφαιρεθούν τα stop
words)
5. Ποιοι είναι οι 10 συγγραφεις με τα περισσότερα βιβλία
6. Ποιοι είναι οι 10 συγγραφείς με τις περισσότερες κριτικές (χρησιμοποιήστε την στήλη
numRatings).
7. Κατατάξτε του συγγραφείς με βάση τα βιβλία τους ανά έτος.
8. Ποιές είναι οι πιο συχνές γλώσσες που έχουν γραφτεί τα βιβλία στα δεδομένα σας
9. Ποιοί είναι οι 10 εκδότες που έχουν εκδώσει τα περισσότερα βιβλία.
10. Έχουν τα βιβλία με τις περισσότερες σελίδες (πχ περισσότερες από 1000 pages)
υψηλότερα ratings ?
11. Συγκεντρώστε σε ένα γράφημα η σε ένα πίνακα όλα τα μοναδικά είδη βιβλίων (genres).
Ποιά είναι τα πιο συχνά genres; Το ίδιο και για τα awards.
12. Φτιάξτε τα wordclouds για τη στήλη description. Σε αυτό το ερώτημα αφαιρέστε τα stop
words, πειραματιστείτε με τις παραμέτρους του wordcloud και εντοπίστε τις πιο
χαρακτηριστικές λέξεις που χρησιμοποιούνται στα βιβλία του συνόλου των δεδομένων
σας.
13. Πόσα βιβλία εκδίδονται ανά έτος ;

In [4]:
import matplotlib.pyplot as plt

# 2) Ποιά είναι τα 10 βιβλία με τις περισσότερες σελίδες.
sorted_by_pages = df_clean.sort_values('pages', key=lambda x: x)
print("Books with most pages: \n", sorted_by_pages['bookId'][:10])

# 3) Ποιά είναι τα 10 βιβλία με τις περισσότερες σελίδες.
sorted_by_rating = df_clean.sort_values('ratingStar5', key=lambda x: x)
print("Books with most 5 star ratings: \n", sorted_by_rating['bookId'][:10])

# 5) Ποιοι είναι οι 10 συγγραφεις με τα περισσότερα βιβλία
grouped_by_author = df_clean.groupby(['author']).count()
sorted_by_author_books = grouped_by_author.sort_values(['bookId'], key=lambda x: x,ascending=False)
print("Authors with most books: \n", sorted_by_author_books['bookId'][:10])

# 6) Ποιοι είναι οι 10 συγγραφείς με τις περισσότερες κριτικές
sorted_by_author_ratings = grouped_by_author.sort_values(['numRatings'], key=lambda x: x,ascending=False)
print("Authors with most ratings: \n", sorted_by_author_ratings['numRatings'][:10])

# 8) Ποιές είναι οι πιο συχνές γλώσσες που έχουν γραφτεί τα βιβλία
grouped_by_language = df_clean.groupby('language')['bookId'].count()
sorted_by_language = grouped_by_language.sort_values(ascending=False)
print("Most popular languages: \n", sorted_by_language[:10])


Books with most pages: 
 11961                        7506252-are-you-kidding-me
14779                   591965.The_Madwoman_of_Chaillot
16039                                    875775.Mission
12187                       131094.Millroy_the_Magician
18666                                      2581254-halo
16150    10394216-the-complete-works-of-charles-dickens
16265                          10168286-the-village-wit
9482                       16312.One_Two_Buckle_My_Shoe
18514                         121646.Murder_in_the_Mews
12052                      7058894-12-stocking-stuffers
Name: bookId, dtype: object
Books with most 5 star ratings: 
 11063            15747245-close-your-eyes-and-see
15460                        2338189.Zany_Hijinx_
10976    12813058-lost-souls---the-cube-of-asgard
11021              13560868-the-language-of-light
13159        14643361-telling-time-by-the-shadows
13790                        2583110-plural-loves
11127                            3454581-the-list
111

## 3) Υλοποίηση Recommendation system (35%)
Ο στόχος ενός τέτοιου συστήματος είναι (1) να προβλέψει τις αξιολογήσεις ενός χρήστη για τα
βιβλία που δεν έχει διαβάσει ακόμα, και (2) να εμφανίσει ένα ταξινομημένη λίστα με τα κορυφαία
Ν βιβλία για τα οποία πιστεύουμε ότι θα ήθελαν να μάθουν περισσότερα. ‘Ενας άλλος στόχος
ενός Recommender είναι (3) να βοηθήσει τους χρήστες να ανακαλύψουν σχετικά βιβλία που δεν
θα είχαν βρει διαφορετικά.
Σε αυτό το ερώτημα θα χρειαστείτε τις στήλες
* BookId
* Description
* Και μόνο όσες γραμμές έχουν γλώσσα “English”. 

Δημιουργήστε τον TF-IDF (Term Frequency - Inverse Document Frequency) πίνακα των
unigrams και των bigrams για τη στήλη description (χρησιμοποιήστε την παράμετρο stop_word
του TfidfVectorizer).

**Cosine Similarity**: Η μετρική αυτή υπολογίζει την ομοιότητα μεταξύ δύο διανυσμάτων x,y,
χρησιμοποιώντας τη γωνία μεταξύ τους (όταν η γωνία είναι 0 σημαίνει ότι τα x και y είναι ίσα , αν
εξαιρέσουμε το μήκος τους). Διατρέξτε τον TF-IDF πίνακα και υπολογίστε το similarity καθενός
βιβλίου με τα υπόλοιπα. Αποθηκεύστε σε ένα python dictionary τα 100 πιο όμοια βιβλία.

Πρόβλεψη: Φτιάξτε μία συνάρτηση η οποία παίρνει σαν είσοδο ένα id και ένα ακέραιο αριθμό N,
και επιστρέφει τα Ν πιο όμοια βιβλία.
recommend(item_id = 4085439, num = 5)

Η έξοδος της συνάρτησης να είναι της παρακάτω μορφής μορφής:
```
Recommending 5 books similar to: The Hunger Games
-------------------------------------------------
Recommended: NAME
Description: DESCRIPTION
(score:0.12235188993161432)
Recommended: NAME
Description: DESCRIPTION
(score:0.12235188993161432 
```


In [5]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Keep only the book Ids and their respective descriptions 
# ΤHN book_descriptions ΧΡΕΙΑΖΕΤΑΙ H recommend() function 
book_descriptions = df_clean.loc[ 
    (df_clean['language'] == 'English'),['bookId','description']].reset_index(drop=True)

vectorizer = TfidfVectorizer(
    token_pattern = r'\b(?![A-Za-z]\b)[A-Za-z]+\b', #ignore numbers and such
    #min_df = 0.0014, # I exclude rare words, as they have little contribution to overarching theme of a book,might change it later
    ngram_range=(1, 2), # unigrams and bigrams
    strip_accents='unicode', # probably won't need that since we are only in English but you never know
    stop_words='english', # ignore stop words 
)

#rows = len(book_descriptions.index)

matrix = vectorizer.fit_transform(book_descriptions['description'])
similarities = cosine_similarity(matrix) # calculate the similarity matrix (735,735)
np.fill_diagonal(similarities,-1) # remove the self similarities
flat = similarities.flatten()
 
# creating pairs with the 100 most similar books
pairs = []
for i in range(len(similarities)):
    array = similarities[i].copy()
    for j in range(100): # for 100 times
        index = np.argmax(array) # get the largest remaining similarity in the flat matrix
        pairs.append((i,index,array[index])) # save the similarity pair
        array[index] = -1 # get remove it from the matrix so as to not find it again
        
dictionary = {}
i = 0
for (index1,index2,sim) in pairs:
    dictionary[i] = (book_descriptions['bookId'][index1],book_descriptions['bookId'][index2],sim)
    i = i+1

def recommend(bookId, findNsimilar):
    title = df_clean.loc[df_clean['bookId'] == bookId, 'title'].values[0]
    print("Recommending 5 books similar to:", title, "\n-------------------------------------------------")
    row = (book_descriptions[book_descriptions['bookId'] == bookId].index.tolist())[0]
    for i in range(findNsimilar):
        bookId = dictionary[row*100+i][1]
        recommended_title = df_clean.loc[df_clean['bookId'] == bookId, 'title'].values[0]
        recommended_description = df_clean.loc[df_clean['bookId'] == bookId, 'description'].values[0]
        print("Recommended: ", recommended_title)
        print("Description: ", recommended_description,"\n")
        print("(score:", dictionary[row*100+i][2], ")\n")
        
recommend('591965.The_Madwoman_of_Chaillot', 100)

## 4) Υλοποίηση Κατηγοριοποίησης (Classification) (35%)
Χρησιμοποιήστε την στήλη genreSingle, βρείτε τα 10 πιο συχνά genres και κρατήστε σε ένα νέο
dataframe τα βιβλία εκείνα που ανήκουν σε αυτες τις 10 πιο συχνές κατηγορίες. Θα χρειαστείτε
το bookId το description και το genreSingle. Στην συνέχεια καθαρίστε την στήλη description
χρησιμοποιώντας τις μεθόδους που είδαμε στα φροντιστήρια (πχ αφαίρεση σημείων στίξης,
μετατροπή όλων των χαρακτήρων σε πεζά, κ.α.). Εφαρμόστε την μέθοδο word2vec για τα
descriptions και στην συνέχεια με την χρήση των embeddings να υπολογίσετε για κάθε
description ένα διάνυσμα με 200-300 τιμές (features) - αυτό θα είναι ο μέσος όρος των
embeddings των λέξεων από τις οποίες αποτελείται το description.

Χρησιμοποιήστε τη βιβλιοθήκη pickle της Python για να αποθηκεύσετε τα χαρακτηριστικά σε
αρχεία *.pkl . Με αυτό τον τρόπο δεν χρειάζεται να υπολογίζονται από την αρχή τα
χαρακτηριστικά κάθε φορά που τρέχετε το πρόγραμμά σας, αλλά μπορείτε μόνο να τα φορτωνεται
στην μνήμη χρησιμοποιώντας την αντίστοιχη μέθοδο load.
Χωρίστε το σύνολο των δεδομένων σε train (80%) και test (20%) χρησιμοποιώντας την μέθοδο
train_test_split() της βιβλιοθήκης sklearn.

Σε αυτό το ερώτημα θα πρέπει το πρόγραμμα σας να μπορεί να βρει τις κατηγορίες (genre) του
συνόλου δοκιμής (test) χρησιμοποιώντας τις παρακάτω μεθόδους Classification:
* Naive Bayes
* Support Vector Machines (SVM, να πειραματιστείτε με τις παραμέτρους kernel (rbf, linear), c
και gamma. H επιλογή των παραμέτρων μπορεί να γίνει και με GridSearchCV)
* Random Forests

Ολα τα παραπάνω μοντέλα θα εκπαιδευτούν MONO στο σύνολο train και θα αξιολογηθούν στο
σύνολο test. Επίσης θα πρέπει να αξιολογήσετε και να καταγράψετε την απόδοση κάθε μεθόδου
χρησιμοποιώντας 10-fold Cross Validation χρησιμοποιώντας τις παρακάτω μετρικές:
* Precision / Recall / F-Measure
* Accuracy

Στο τέλος να ετοιμάσετε ένα πίνακα με τα αποτελέσματα των πειραμάτων σας για κάθε
μετρική/παράμετρο που χρησιμοποιήσατε.

In [11]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from  tensorflow.keras.preprocessing.text

df_genre_counts = df_clean.groupby('genreSingle')[['bookId']].count()
top_10 = df_genre_counts.sort_values(by='bookId', ascending=False)[:10]
top_10_with_genre = top_10.reset_index()
print(np.array(top_10_with_genre['genreSingle']))

# DataFrame.drop(labels=None, *, axis=0, index=None, columns=None, 
#                level=None, inplace=False, errors='raise')

new_df = df_clean.drop(df_clean[~df_clean['genreSingle'].isin(top_10_with_genre['genreSingle'])].index)
new_df = new_df.loc[ 
    (new_df['language'] == 'English')].reset_index(drop=True)

vectorizer = TfidfVectorizer(
    stop_words='english', # ignore stop words 
)

clean_descriptions = []
for description in new_df['description']:
    vectorizer.fit(description.split('\n'))
    clean_descriptions.append(vectorizer.get_feature_names_out())
    



SyntaxError: ignored